Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [3]:
train.shape, test.shape

((59400, 41), (14358, 40))

In [4]:
# import library
import numpy as np
# wrangling data on all train and test sets

def wrangle(data):
    '''Wrangle both test and train sets'''
    
    # prevent copy warning
    data = data.copy()
    
    # replace value close to zero with zeros for latitide column
    data['latitude'] = data['latitude'].replace(-2e-08, 0)
    
    # replacing zeros with nan values
    # impute with missing values later in the pipeline
    cols_with_zeros = ['latitude', 'longitude', 'population', 'amount_tsh', 'construction_year', 'num_private', 'gps_height']
    for col in cols_with_zeros:
        data[col] = data[col].replace(0, np.NaN)
        
    return data

In [5]:
# apply the function to wrangle the train and test set
train = wrangle(train)
test = wrangle(test)

train.shape, test.shape

((59400, 41), (14358, 40))

In [6]:
print(train.isnull().sum())

id                           0
amount_tsh               41639
date_recorded                0
funder                    3635
gps_height               20438
installer                 3655
longitude                 1812
latitude                  1812
wpt_name                     0
num_private              58643
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population               21381
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year        20709
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [7]:
# the target for this dataset
target = 'status_group'

# drop duplicates
duplicate_column = (['quantity_group', 'quality_group', 
                    'waterpoint_type_group', 'extraction_type_group', 
                    'source_type', 'payment_type'])
# drop both always constant and all random columns
noneed_columns = ['recorded_by', 'id']
# adding both drop list in a new variable
unusuable_columns = duplicate_column + noneed_columns

# get a dataframe with all features except the target
train_features = train.drop([target] + unusuable_columns, axis=1)

# get a list of numeric columns
numerics_features = train_features.select_dtypes(include='number').columns.to_list()

# get a list of non-numeric columns
categorical_features = train_features.select_dtypes(exclude='number').columns.to_list()

# combine the numeric and non-numeric features
features = numerics_features + categorical_features
features

['amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'region_code',
 'district_code',
 'population',
 'construction_year',
 'date_recorded',
 'funder',
 'installer',
 'wpt_name',
 'basin',
 'subvillage',
 'region',
 'lga',
 'ward',
 'public_meeting',
 'scheme_management',
 'scheme_name',
 'permit',
 'extraction_type',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'water_quality',
 'quantity',
 'source',
 'source_class',
 'waterpoint_type']

In [8]:
# train_features.select_dtypes(include='number').columns.to_list()

In [9]:
# target = 'status_group'

In [10]:
# train_features = train.columns.drop(target)
# train_features

In [12]:
# arranging data in X features matrix and y response vector
X_train = train[features]
y_train = train[target]
X_test = test[features]

In [13]:
X_train.shape, y_train.shape, X_test.shape

((59400, 32), (59400,), (14358, 32))

In [19]:


from scipy.stats import randint, uniform
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier()
)

param_distributions = {   
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestclassifier__n_estimators': randint(100, 500),
    'randomforestclassifier__min_samples_leaf': randint(1, 20),
#     'randomforestclassifier__max_samples_leaf': randint(80, 100),
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0, 1),
}

search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=100, 
    cv=5, 
    scoring='accuracy', 
    verbose=10, 
    random_state=42, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   57.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed: 14.7min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed: 20.6min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 22.1min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 25.4min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 29.1min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 34.9min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 39.7min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed: 43.3min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 46

In [20]:
print('Best hyperparameters:', search.best_params_)

Best hyperparameters: {'randomforestclassifier__max_depth': None, 'randomforestclassifier__max_features': 0.6117207462343522, 'randomforestclassifier__min_samples_leaf': 3, 'randomforestclassifier__n_estimators': 175, 'simpleimputer__strategy': 'median'}


In [21]:
search.best_score_

0.8093434343434343

In [22]:
pipeline = search.best_estimator_

y_pred = pipeline.predict(X_test)
y_pred

array(['non functional', 'functional', 'functional', ..., 'functional',
       'functional', 'non functional'], dtype=object)

In [24]:
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('nirmal_kaggle_U2_M3_S1-wed.csv', index=False)